In [1]:
!source activate py36

In [2]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [3]:
root = "/home/faske/g/balsam"

In [4]:
cd $root

/working/tfaske/balsam


In [10]:
!mkdir SNPcall
!mkdir SNPcall/bowtie2

In [5]:
pwd

'/working/tfaske/balsam'

# Actual Mapping 
 

In [6]:
fastq_files = []
for seqdir in ['demult/fastq/']:
    os.chdir('{}/{}'.format(root, seqdir))
    files = !find . -name '*fq.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [7]:
len(fastq_files)

373

In [8]:
cd $root

/working/tfaske/balsam


In [14]:
assembly = "/home/faske/g/balsam/assembly/reference.fasta"

#indexed as: bowtie2-build --threads 10 -f reference.fasta reference.fasta

In [15]:
# --very-fast-local
# Same as: -D 5 -R 1 -N 0 -L 25 -i S,1,2.00

# --fast-local
# Same as: -D 10 -R 2 -N 0 -L 22 -i S,1,1.75

# --sensitive-local
# Same as: -D 15 -R 2 -N 0 -L 20 -i S,1,0.75 (default in --local mode)

# --very-sensitive-local
# Same as: -D 20 -R 3 -N 0 -L 20 -i S,1,0.50

#@lview.remote()
def run_bowtie2(args):
    import os, multiprocessing, socket #stopwatch,
    #timer = stopwatch.Timer()
    cpus = 12
    assembly, reads, outdir = args
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(reads)))
    #sam = tempfile.NamedTemporaryFile(delete=False, dir="/tmp")
    #print (sam)
    cmd = "bowtie2 --local -D 20 -R 3 -N 1 -L 20 -i S,1,0.50 -p %d -x %s -U %s -S %s" % (cpus,
                                                               assembly,
                                                               reads,
                                                               sam)
    res = None
    res = cmd
#     if not os.path.exists(sam):
#         res = !$cmd
    #timer.stop()
    return  cmd #,timer.elapsed

In [16]:
bowtie_dir = os.path.join(root,"SNPcall/bowtie2/")
assert(bowtie_dir)

In [17]:
res = []
for f in fastq_files:
    r = run_bowtie2((assembly, f, bowtie_dir))
    res.append(r)

In [18]:
res[0]

'bowtie2 --local -D 20 -R 3 -N 1 -L 20 -i S,1,0.50 -p 12 -x /home/faske/g/balsam/assembly/reference.fasta -U /working/tfaske/balsam/demult/fastq/BS_AN_1.F.fq.gz -S /home/faske/g/balsam/SNPcall/bowtie2/BS_AN_1.F.fq.gz.sam'

In [19]:
pwd

'/working/tfaske/balsam'

In [20]:
def write_bowtie_sh(cmds):
    with open("SNPcall/run_bowtie.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o bowtie2.out
#$ -e bowtie2.err
source activate py36\n""")
        for elem in cmds:
            o.write("{}\n".format(elem))

In [21]:
cd $root

/working/tfaske/balsam


In [22]:
write_bowtie_sh(res)

# Run run_bowie.sh
    cd /home/faske/g/piper/retic/SNPcall 
    bash run_bowtie.sh &> bowtie.out &
    disown -h
    

In [35]:
fqs = !find /home/faske/g/balsam/demult/fastq/ -name "*fq.gz"

In [36]:
sams = !find $bowtie_dir -name "*.sam"

In [37]:
len(fqs), len(sams)

(373, 373)

In [38]:
sams

['/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_1.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_10.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_11.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_12.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_2.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_3.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_4.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_5.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_6.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_7.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_8.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_9.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AS_1.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AS_10.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AS_11.F.fq.gz.sam',
 '/home/faske/g/balsam/SNPcall/bowtie2/BS_AS_12.F.fq.gz.sam',
 '/home/faske/g/ba

In [39]:
SNPcall_dir = os.path.join(root,'SNPcall')
assert(SNPcall_dir)

In [40]:
cd $SNPcall_dir

/working/tfaske/balsam/SNPcall


In [41]:
def write_sam2bam_sh(sam):
    with open("./run_sam2bam.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o sam2bam.out
#$ -e sam2bam.err
source activate py36\n""")
        for s in sam:
            bam = s.replace(".sam", ".bam")
            bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
            o.write("samtools view -b {0} -o {1}\nsamtools sort -@ 16 {1} -o {2}\nsamtools index {2}\n".format(s,bam,bam_sorted))

In [42]:
write_sam2bam_sh(sams)

# run_sam2bam.sh
    cd /working/tfaske/piper/retic/SNPcall 
    bash run_sam2bam.sh &> sam2bam.out & 
    disown -h 
    close window


In [38]:
#u.get_async_progress(sam_bam_jobs)
#sam_bam_jobs

SyntaxError: invalid syntax (<ipython-input-38-3c875eecaf82>, line 3)

In [43]:
cd $bowtie_dir

/working/tfaske/balsam/SNPcall/bowtie2


In [48]:
sorted_bams = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    sorted_bams.append(x)
sorted_bams = sorted(sorted_bams)

#assert len(sorted_bams) == len(fastq_files)

len(sorted_bams), len(fqs), len(sams)
#missing 3 bam files from lane1

(373, 373, 373)